# Import packages

In [ ]:
import pandas as pd
from teradataml import *

from modules.db import TeradataDatabase
from utils import clean_text, load_embedding_model
from constants import TEST_DATA_PATH, E5_LARGE_INSTRUCT_CONFIG_PATH
from modules.models import SentenceEmbeddingModel, SentenceEmbeddingConfig

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\os255022\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Log directory cleaned: c:\internship\AMuRD-Iteration-7\src\logs
Logger initialized. All logs will be saved to: c:\internship\AMuRD-Iteration-7\src\logs\borai_20250814_141000.log


## Connect to database

In [ ]:
td_db = TeradataDatabase()
td_db.connect()


## Load data to database

In [ ]:
df = pd.read_csv(TEST_DATA_PATH)
df.head()

In [ ]:
df = df[["Item_Name", "Brand"]]
df.rename(columns={"Item_Name": "product_name", "Brand": "brand_name"}, inplace=True)

In [ ]:
df.drop_duplicates(subset=["product_name"], inplace=True)
df.dropna(subset=["product_name"], inplace=True)
df

In [ ]:
df["brand_name"] = df["brand_name"].astype(str)

In [ ]:
tdf = DataFrame(df)

In [ ]:
tdf = tdf.assign(product_id=tdf.index_label)
tdf = tdf.drop(columns=["index_label"])
tdf

In [ ]:
tdf = tdf.select(["product_id", "product_name", "brand_name"])
tdf

In [ ]:
copy_to_sql(tdf, "products", "amurd", if_exists="replace")

## Load table from database

In [ ]:
tdf = td_db.execute_query("SELECT * FROM amurd.products")
tdf = DataFrame(tdf, False)
tdf

In [4]:
model = load_embedding_model(E5_LARGE_INSTRUCT_CONFIG_PATH)

In [ ]:
import json
df = tdf.select(["product_name", "product_id"]).to_pandas()
product_id = df["product_id"].tolist()[:20]
product_names = df["product_name"].tolist()[:20]

embeddings = model.get_embeddings(product_names, "query")
embeddings_bytes = [json.dumps(embedding.tolist()) for embedding in embeddings]
embeddings_bytes

In [ ]:
df_new = pd.DataFrame({
    "product_id": product_id,
    "embedding": embeddings_bytes
})

tdf_new = DataFrame(df_new, types={"embedding": VARCHAR()})
# copy_to_sql(df_new, table_name="product_embeddings", if_exists="replace", types={"embedding": BLOB(4096)})

In [ ]:
tdf_new

In [ ]:
tdf_new.dtypes

In [ ]:
s = tdf_new.to_pandas()

In [35]:
embedding = s["embedding"].tolist()

In [36]:
embedding[0]

'[0.00963592529296875, 0.02484130859375, 0.0007576942443847656, -0.05615234375, 0.01186370849609375, -0.028076171875, -0.00777435302734375, 0.0180206298828125, 0.02496337890625, -0.0257720947265625, 0.024505615234375, 0.00817108154296875, -0.032196044921875, -0.0125274658203125, -0.0224761962890625, -0.0284271240234375, -0.033660888671875, 0.0323486328125, -0.00598907470703125, -0.01001739501953125, 0.034820556640625, -0.0089569091796875, -0.04022216796875, -0.0391845703125, -0.017730712890625, 0.004497528076171875, -0.03338623046875, -0.031982421875, -0.049072265625, -0.004749298095703125, -0.01049041748046875, 0.0243072509765625, -0.0310211181640625, -0.0638427734375, -0.028167724609375, 0.053955078125, 0.037445068359375, 0.0140228271484375, -0.03759765625, 0.035003662109375, -0.004634857177734375, 0.027862548828125, 0.01702880859375, -0.038299560546875, -0.02203369140625, 0.032440185546875, 0.021697998046875, 0.005260467529296875, 1.8596649169921875e-05, 0.0474853515625, 0.036132812

In [ ]:
import json
json.loads(embedding[0])

1024

## Disconnect

In [ ]:
td_db.disconnect()